In [1]:
%load_ext autoreload
%autoreload 2

import sscmap

In [2]:
meta = RecordMeta(doi="10.1016/j.cell.2018.08.039")

In [3]:
meta.set(
    technology="MIBI",
    species="human",
    tissue="breast",
    disease="cancer",
    disease_subtype="breast cancer",)